In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.serif'] = ['SimHei']

import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

# 航班数据
fileName = open(r"D:/Python/毕设实验部分/TUYU/data/final_feature.csv")
feature = pd.read_csv(fileName)
feature.head()

,航班编号,计划起飞时间,计划到达时间,计划起飞时刻,计划到达时刻,计划飞行时间,起飞间隔,前序延误,平均延误时间,最大延误时间,...,本时段实际进港延误航班数,本时段离港延误时长,本时段离港平均延误时长,本时段离港延误率,上一时段离港平均延误时长,上一时段进港延误时长,上一时段离港延误时长,上一时段实际进港航班数,上一时段进港延误率,标签
0,CZ3855,16801,16801,0,3,3.666667,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,CZ3855,16801,16801,0,3,3.666667,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,CZ3855,16801,16801,0,1,1.250000,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,CZ3855,16801,16801,0,1,1.250000,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,CZ3295,16801,16801,0,1,1.166667,NaN,NaN,0.324175,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [2]:
del(feature['本时段离港平均延误时长'])
del(feature['上一时段离港平均延误时长'])
# 1.前序延误 = 前序航班 实际到达时间 - 计划到达时间
# 2.起飞间隔 = 当前航班的计划起飞时间 - 前序航班实际到达时间
feature['起飞间隔'] = feature['起飞间隔'].fillna(0)
feature['前序延误'] = feature['前序延误'].fillna(0)
feature['出发机场天气得分'] = feature['出发机场天气得分'].interpolate()
feature['到达机场天气得分'] = feature['到达机场天气得分'].interpolate()
feature['平均延误时间'] = feature['平均延误时间'].interpolate()
feature['最大延误时间'] = feature['最大延误时间'].interpolate()
feature['延误时间中位数'] = feature['延误时间中位数'].interpolate()
feature['延误时间标准差'] = feature['延误时间标准差'].interpolate()

feature['本时段计划离港航班数'] = feature['本时段计划离港航班数'].fillna(feature['本时段计划离港航班数'].mean())
feature['本时段实际离港航班数'] = feature['本时段实际离港航班数'].fillna(feature['本时段实际离港航班数'].mean())
feature['本时段进港延误时长'] = feature['本时段进港延误时长'].fillna(feature['本时段进港延误时长'].mean())
feature['本时段实际进港延误航班数'] = feature['本时段实际进港延误航班数'].fillna(feature['本时段实际进港延误航班数'].mean())
feature['本时段离港延误时长'] = feature['本时段离港延误时长'].fillna(feature['本时段离港延误时长'].mean())
feature['本时段离港延误率'] = feature['本时段离港延误率'].fillna(feature['本时段离港延误率'].mean())
feature['上一时段进港延误时长'] = feature['上一时段进港延误时长'].fillna(feature['上一时段进港延误时长'].mean())
feature['上一时段离港延误时长'] = feature['上一时段离港延误时长'].fillna(feature['上一时段离港延误时长'].mean())
feature['上一时段实际进港航班数'] = feature['上一时段实际进港航班数'].fillna(feature['上一时段实际进港航班数'].mean())
feature['上一时段进港延误率'] = feature['上一时段进港延误率'].fillna(feature['上一时段进港延误率'].mean())

In [3]:
from sklearn import metrics
def ks(y_predicted, y_true):
    label=y_true
    #label = y_true.get_label()
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()

In [8]:
#LGBM模型结果
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dataset = feature.as_matrix()
X = dataset[:,1:36]
Y = dataset[:,36]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)


model = lgb.LGBMClassifier(
        boosting_type = 'gbdt',
        boosting = 'dart',
        learning_rate= 0.3,
        num_leaves = 25,
        max_depth=15,
        min_data_in_leaf= 25,
        feature_fraction= 0.8,
        bagging_fraction= 1,
        bagging_freq= 1,
        lambda_l1= 1,
        lambda_l2= 1,
        min_split_gain=0.5,
        objective= 'binary',
        metric= 'auc',
        is_unbalance=True
#         scale_pos_weight=10
    )


lgb_model = model.fit(x_train,  # 训练集
                      y_train.astype('int'))  # 早停系数





#利用训练的模型来测试  训练集
prediction_train = lgb_model.predict(x_train)
prediction_train_predprob = lgb_model.predict_proba(x_train)[:,1]
 
# #利用训练的模型来测试  验证集集
prediction_test = lgb_model.predict(x_test)
prediction_test_predprob = lgb_model.predict_proba(x_test)[:,1]

prediction_test = prediction_test.tolist()
y_test = y_test.tolist()
prediction_test = np.array(prediction_test)
y_test = np.array([i for i in y_test])

y_train = y_train.tolist()
y_train = np.array([i for i in y_train])

prediction_train = prediction_train.tolist()
prediction_train = np.array(prediction_train)


print("LightGBM模型结果")

testAccuracy = metrics.accuracy_score(y_train, prediction_train)
print("训练样本的准确率: %.4f" % testAccuracy)
 
average_precision = metrics.average_precision_score(y_train, prediction_train)
print("average_precision: %.4f" % average_precision)
 
#查准率
averagePrecisionScore = metrics.precision_score(y_train, prediction_train)
print("查准率: %.4f" % averagePrecisionScore)
 
##召回率
returnResultScore = metrics.recall_score(y_train, prediction_train)
print("召回率: %.4f" % returnResultScore)
 
##F1值
F1Score = metrics.f1_score(y_train, prediction_train)
print("F1-Score: %.4f" % F1Score)
 
#计算auc
roc_auc_predprob = metrics.roc_auc_score(y_train, prediction_train_predprob)
print("AUC-Score_predprob: %.4f" % roc_auc_predprob)
 

testAccuracy = metrics.accuracy_score(y_test, prediction_test)
print("验证样本的准确率: %.4f" % testAccuracy)
 
#查准率
averagePrecisionScore = metrics.precision_score(y_test, prediction_test)
print("查准率: %.4f" % averagePrecisionScore)
 
##召回率
returnResultScore = metrics.recall_score(y_test, prediction_test)
print("召回率: %.4f" % returnResultScore)
 
##F1值
F1Score = metrics.f1_score(y_test, prediction_test)
print("F1-Score: %.4f" % F1Score)
 
#计算auc
roc_auc = metrics.roc_auc_score(y_test, prediction_test_predprob)
print("AUC-Score: %.4f" % roc_auc)
 
ks_score = ks(prediction_test, y_test)
print(ks_score) 

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
LightGBM模型结果
训练样本的准确率: 0.9351
average_precision: 0.4606
查准率: 0.5327
召回率: 0.8435
F1-Score: 0.6530
AUC-Score_predprob: 0.9549
验证样本的准确率: 0.9348
查准率: 0.5305
召回率: 0.8417
F1-Score: 0.6508
AUC-Score: 0.9533
('ks', 0.7837036575825952)

In [16]:
from sklearn.metrics import classification_report 
print("LightGBM模型评估")
print(classification_report(prediction_test, y_test))

LightGBM模型评估
              precision    recall  f1-score   support

           0       0.94      0.99      0.96   1352715
           1       0.84      0.53      0.65    175005

    accuracy                           0.93   1527720
   macro avg       0.89      0.76      0.81   1527720
weighted avg       0.93      0.93      0.93   1527720



In [17]:
import pickle
f= open('D:/Python/毕设实验部分/TUYU/model/model/lgb_model.pickle','wb')
pickle.dump(lgb_model,f)
f.close()

In [19]:
lgb_offline=(prediction_test_predprob >= 0.7) *1
from sklearn.metrics import classification_report 
print(classification_report(lgb_offline, y_test))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98   1401116
           1       0.78      0.68      0.73    126604

    accuracy                           0.96   1527720
   macro avg       0.88      0.83      0.85   1527720
weighted avg       0.96      0.96      0.96   1527720

